# VacationPy

In [1]:
import sys
sys.path.append("..")

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import time

# Import API key
from api_keys import vacation_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
output_data_file = "output_data/cities.csv"
VacationPy = pd.read_csv('output_data/cities.csv',index_col='City')
VacationPy.reset_index(inplace=True)
VacationPy

,City,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,ostrovnoy,0,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,mahebourg,1,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,qaanaaq,2,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,zhuhai,3,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,cape town,4,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,yabrud,543,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,paraiso,544,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,veraval,545,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,novyy urgal,546,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure GMaps
gmaps.configure(api_key=vacation_api_key)

In [12]:
#Weight using Lat and Lng
locations = VacationPy[["Lat", "Lng"]].astype(float)
weights = VacationPy["Humidity"]

fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

#Add Heatmap Layer
heat_layer = gmaps.heatmap_layer(locations,weights = weights,
                              dissipating=False, max_intensity=100,
                              point_radius = 2)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions:
* A max temperature higher than 60 degrees but lower than 75 degrees
* Wind speed less than 10 m/s
* Zero Cloudiness
* Drop any rows will null values.

In [6]:
perfect_weather = VacationPy[(VacationPy['Max Temp'] >=60) &
                             (VacationPy['Max Temp'] <=75) &
                             (VacationPy['Wind Speed'] <10) &
                             (VacationPy['Cloudiness'] ==0)].dropna().reset_index(drop=True)
perfect_weather

,City,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,varhaug,37,0,NO,1558378761,63,58.61,5.65,64.99,8.05
1,ponta do sol,70,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
2,emerald,84,0,AU,1558378770,82,-23.53,148.16,60.80,6.93
3,sanchazi,141,0,CN,1558378781,82,30.43,103.62,64.40,2.24
4,straumen,168,0,NO,1558378786,52,63.87,11.30,68.00,9.17
5,port hedland,192,0,AU,1558378791,51,-20.31,118.58,62.60,6.93
6,mount isa,214,0,AU,1558378794,56,-20.73,139.49,68.00,6.93
7,hami,259,0,CN,1558378834,11,42.84,93.51,69.72,6.20
8,mogok,297,0,MM,1558378844,51,22.92,96.51,72.24,2.77
9,mikkeli,298,0,FI,1558378844,59,61.69,27.27,62.60,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = perfect_weather
hotel_df.dropna()
hotel_df["Hotel Name"] = ""
hotel_df

,City,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,varhaug,37,0,NO,1558378761,63,58.61,5.65,64.99,8.05,
1,ponta do sol,70,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
2,emerald,84,0,AU,1558378770,82,-23.53,148.16,60.80,6.93,
3,sanchazi,141,0,CN,1558378781,82,30.43,103.62,64.40,2.24,
4,straumen,168,0,NO,1558378786,52,63.87,11.30,68.00,9.17,
5,port hedland,192,0,AU,1558378791,51,-20.31,118.58,62.60,6.93,
6,mount isa,214,0,AU,1558378794,56,-20.73,139.49,68.00,6.93,
7,hami,259,0,CN,1558378834,11,42.84,93.51,69.72,6.20,
8,mogok,297,0,MM,1558378844,51,22.92,96.51,72.24,2.77,
9,mikkeli,298,0,FI,1558378844,59,61.69,27.27,62.60,6.93,


In [ ]:
#set params
params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": vacation_api_key}

In [ ]:
for index, row in perfect_weather.iterrows():
    params["location"] = f"{row['Lat']}, {row['Lng']}"
    lat=row["Lat"]
    lng=row["Lng"]
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params)
    response = hotels.json()
    
    try:
        hotel_df.loc[index,"Hotel Name"]=response['results'][0]['name']
        
    except (KeyError, IndexError):
        print("skipping...")
        
    #wait 1 sec to make another api request
    time.sleep(1)
        
pprint(response)

skipping...
skipping...


In [9]:
hotel_df

,City,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,varhaug,37,0,NO,1558378761,63,58.61,5.65,64.99,8.05,
1,ponta do sol,70,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
2,emerald,84,0,AU,1558378770,82,-23.53,148.16,60.80,6.93,
3,sanchazi,141,0,CN,1558378781,82,30.43,103.62,64.40,2.24,
4,straumen,168,0,NO,1558378786,52,63.87,11.30,68.00,9.17,
5,port hedland,192,0,AU,1558378791,51,-20.31,118.58,62.60,6.93,
6,mount isa,214,0,AU,1558378794,56,-20.73,139.49,68.00,6.93,
7,hami,259,0,CN,1558378834,11,42.84,93.51,69.72,6.20,
8,mogok,297,0,MM,1558378844,51,22.92,96.51,72.24,2.77,
9,mikkeli,298,0,FI,1558378844,59,61.69,27.27,62.60,6.93,


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))